Face Recognation with MTCNN

In [1]:
import cv2
import numpy as np
import torch
from facenet_pytorch import InceptionResnetV1, MTCNN
from sklearn.metrics.pairwise import cosine_similarity
from torchvision import transforms
from PIL import Image
import os

# Load the face recognition model (InceptionResnetV1 pre-trained on VGGFace2)
facenet_model = InceptionResnetV1(pretrained='vggface2').eval()

# Load MTCNN for face detection
mtcnn = MTCNN(keep_all=True, min_face_size=40)  # Keep only face-sized detections

# Preprocessing for the face images
preprocess = transforms.Compose([
    transforms.Resize((160, 160)),  # InceptionResnetV1 expects 160x160 input images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Function to extract features from an image
def extract_features(image):
    try:
        # Ensure image is RGB (remove alpha channel if present)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        img = preprocess(image)
        img = torch.unsqueeze(img, 0)  # Add batch dimension
        with torch.no_grad():
            feature = facenet_model(img).numpy()
        return feature
    except Exception as e:
        print(f"Error extracting features: {e}")
        return None

# Function to compute cosine similarity
def compute_similarity(feature1, feature2):
    return cosine_similarity(feature1.reshape(1, -1), feature2.reshape(1, -1)).flatten()[0]

# Identify a person given a query feature vector
def identify_person(query_feature, feature_database, threshold=0.75):
    try:
        similarities = {}
        for person_name, person_feature in feature_database.items():
            similarity = compute_similarity(query_feature, person_feature)
            similarities[person_name] = similarity

        identified_person = max(similarities, key=similarities.get)
        highest_similarity = similarities[identified_person]

        if highest_similarity < threshold:
            return "Unknown", highest_similarity
        else:
            return identified_person, highest_similarity
    except Exception as e:
        print(f"Error identifying person: {e}")
        return "Unknown", 0.0

# Load the feature database for known persons
def create_feature_database(train_folder):
    feature_database = {}
    for person_name in os.listdir(train_folder):
        person_folder = os.path.join(train_folder, person_name)
        if os.path.isdir(person_folder):
            images = [Image.open(os.path.join(person_folder, filename)) for filename in os.listdir(person_folder)]
            features = np.array([extract_features(img) for img in images if extract_features(img) is not None])
            if features.size > 0:
                feature_database[person_name] = np.mean(features, axis=0)
    return feature_database

# Paths to the train folder
# data from Labelled Faces in the Wild (LFW) Dataset + 4 collegues photo from internet
train_folder = 'train'  # Path to the folder containing training images

# Create the feature database
train_feature_database = create_feature_database(train_folder)

# Start the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break
    
    # Detect faces using MTCNN
    try:
        boxes, _ = mtcnn.detect(frame)
        
        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                
                # Extract the face region
                face = frame[y1:y2, x1:x2]
                face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
                
                # Extract features from the detected face
                face_feature = extract_features(face_pil)
                
                if face_feature is not None:
                    # Identify the person
                    identified_person, similarity = identify_person(face_feature, train_feature_database, threshold=0.4)
                    
                    # Draw a rectangle around the face and put the name
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, f'{identified_person} ({similarity:.2f})', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    except Exception as e:
        print(f"Error during face detection/processing: {e}")
    
    # Display the frame with detections and recognition
    cv2.imshow('Live Face Recognition', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the capture and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-03 14:33:57.033 Python[4343:171007] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
